DEFAULT LIBRARIES

import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt

# SIMPLE LINEAR REGRESSION MODEL


Linear_model= smf.ols(formula= "DEPENDENT_VARIABLE~INDEPENDENT", data=DF)
model = Linear_model.fit()
print(model.summary())

### CORRELATION AND R2 (Explained Variation)

Finding the Correlation and Idenfiying the P:
    df.corrwith(df.dependent_variable).abs().sort_values(ascending = False)

If we want to find Corrleation of all Variables:
    df.corr() 

If asked for pair plot: 
    sns.pairplot(df)

 R2 - from the model 
      - explaination = The value of  R-squared is 0.516, which means that 51.6% of the variations in the dependent variable is explained by the independet varaible. 

CORRELATION AND FINDING R2 
    Find the correlation between `Petrol_consumption` and the rest of the variables in *petrol_consumption_train.csv*. Based on the correlations, a simple linear regression model with which predictor will have the least R-squared value for predicting `Petrol_consumption`. Don't develop any linear regression models.

        train.corrwith(train.Petrol_consumption)
        SQUARE THE VALUES


### PREDICTION AND CONFIDENCE INTERVAL

AT SPECIFIC POINTS 
    Using model to estimate petrol consumption for a state in which 50% of the population has a driver’s license.
    Confidence and prediction intervals for your estmate? Which interval includes the irreducible error?

        prediction = model.get_prediction(pd.DataFrame({'Prop_license':[0.5]})) # if 10% put 0.1
        prediction.summary_frame(alpha=0.05)

        Confidence Interval= (mean_ci_lower, mean_ci_upper)
        Prediction Interval= (obs_ci_lower, obs_ci_upper)

 FIND INTERVALS 
        intervals = model.get_prediction(test)
        intervals.summary_frame(alpha=0.05)
        interval_table = intervals.summary_frame(alpha=0.05)

Variable significance from confidence interval
    conf = model.conf_int(alpha=0.05)
    print(conf)


### RESIDUAL STANDARD ERROR (RSE)The standard deviation of the irreducible error

    np.sqrt(model.mse_resid)

### RMSE & PREDICTION

    PREDICTION
        test= pd.read_csv("petrol_consumption_test.csv")
        pred_price = model.predict(test['Prop_license'])

    RMSE
        np.sqrt(((test.Petrol_consumption - pred_price)**2).mean())

### OVERFITTING & UNDERFITTING 


    train_pred_price = model.predict(train['Prop_license'])
    np.sqrt(((train.Petrol_consumption - train_pred_price)**2).mean())
    Compare train RMSE and test RMSE 

    ORRR

    CHECK IF: 
        residual standard error on the training data is close to the RMSE on the test data: not overfitting
        In case we  a model on the training data, its performance on unknown data is likely to be worse than that on the training data.

### PLOTTING THE GRAPH & INTERVALS 

Visualize the regression line
    sns.regplot(x = 'engineSize', y = 'price', data = train, color = 'orange',line_kws={"color": "red"})
    plt.xlim(-1,7)

    FIND INTERVALS 
        intervals = model.get_prediction(test)
        intervals.summary_frame(alpha=0.05)
        interval_table = intervals.summary_frame(alpha=0.05)
    PLOT THE GRAPH
        (VARIABLE RELATION) sns.scatterplot(x = test.Prop_license, y = test.Petrol_consumption,color = 'orange', s = 10) 
        (REGRESSION LINE) sns.lineplot(x = test.Prop_license, y = pred_price, color = 'red')
        (CONFIDENCE INTERVAL)sns.lineplot(x = test.Prop_license, y = interval_table.mean_ci_lower, color = 'blue')
        (CONFIDENCE INTERVAL)sns.lineplot(x = test.Prop_license, y = interval_table.mean_ci_upper, color = 'blue' label='_nolegend_')
        (PREDICTION INTERVAL)sns.lineplot(x = test.Prop_license, y = interval_table.obs_ci_lower, color = 'green')
        (PREDICTION INTERVAL)sns.lineplot(x = test.Prop_license, y = interval_table.obs_ci_upper, color = 'green')
        plt.legend(labels=["Regression line","Confidence interval", "Prediction interval"])

COMPARING (Actual vs Predicted)
    Make a visualization that compares the predicted car prices with the actual car prices
        sns.scatterplot(x = testp.price, y = pred_price)
        #In case of a perfect prediction, all the points must lie on the line x = y.
        sns.lineplot(x = [0,testp.price.max()], y = [0,testp.price.max()],color='orange') #Plotting the line x = y.
        plt.xlabel('Actual price')
        plt.ylabel('Predicted price')

# Multiple Linear Regression: (MLR)

MODEL 
#Using the ols function to create an ols object. 'ols' stands for 'Ordinary least squares'
    ols_object = smf.ols(formula = 'price~year+mileage+mpg+engineSize', data = train)
    model = ols_object.fit()
    model.summary()
    

### Prediction


data = {'lcavol': [1.35],'age' : 65,'lweight': [3.65],'lweight' : [3.65],'lbph' : [0.1],'svi' : 0.22,'lcp' : -0.18,
       'gleason': 6.75,
       'pgg45' : 25}
dataframe = pd.DataFrame(data)
prediction = model.predict(dataframe)
print(prediction)

Interval 
    intervals = model.get_prediction(dataframe)
    intervals.summary_frame(alpha=0.05)

### Plotting the Residual Point 

sns.scatterplot(x = model.fittedvalues, y=model.resid,color = 'orange')
sns.lineplot(x = [pred_price.min(),pred_price.max()],y = [0,0],color = 'blue')
plt.xlabel('Predicted price')
plt.ylabel('Residual')


# Variable interaction

#Considering interaction between engineSize and year
    ols_object = smf.ols(formula = 'price~year*engineSize+mileage+mpg', data = train)
    model = ols_object.fit()
    model.summary()

#### Qualitative Model

Changing BaseLines (Qualitative Model)
    ols_object = smf.ols(formula = 'price~engineSize+C(transmission, Treatment("Manual"))', data = train_updated)
    model = ols_object.fit()
    model.summary()

Visualizing the developed model that has a qualitative variable


    plt.rcParams["figure.figsize"] = (9,6)
    sns.set(font_scale = 1.3)
    x = np.linspace(train_updated.engineSize.min(),train_updated.engineSize.max(),100)
    ax = sns.lineplot(x = x, y = model.params['engineSize']*x+model.params['Intercept'], color = 'red')
    sns.lineplot(x = x, y = model.params['engineSize']*x+model.params['Intercept']+model.params['transmission[T.Semi-Auto]'], color = 'blue')
    sns.lineplot(x = x, y = model.params['engineSize']*x+model.params['Intercept']+model.params['transmission[T.Manual]'], color = 'green')
    plt.legend(labels=["Automatic","Semi-Automatic", "Manual"])
    plt.xlabel('Engine size (in litre)')
    plt.ylabel('Predicted car price')
    ax.yaxis.set_major_formatter('${x:,.0f}')

Visualizing the interaction between a qualitative and quantitave variable 

    plt.rcParams["figure.figsize"] = (9,6)
    sns.set(font_scale = 1.3)
    x = np.linspace(train_updated.engineSize.min(),train_updated.engineSize.max(),100)
    ax = sns.lineplot(x = x, y = model.params['engineSize']*x+model.params['Intercept'], label='Automatic', color = 'red')
    plt.plot(x, (model.params['engineSize']+model.params['engineSize:transmission[T.Semi-Auto]'])*x+model.params['Intercept']+model.params['transmission[T.Semi-Auto]'], '-b', label='Semi-Automatic')
    plt.plot(x, (model.params['engineSize']+model.params['engineSize:transmission[T.Manual]'])*x+model.params['Intercept']+model.params['transmission[T.Manual]'], '-g', label='Manual')
    plt.legend(loc='upper left')
    plt.xlabel('Engine size (in litre)')
    plt.ylabel('Predicted car price')
    ax.yaxis.set_major_formatter('${x:,.0f}')

### Quadratic transformation

Let us transform the predictor mileage to include (i.e., perform a quadratic transformation on the predictor).
#Including mileage squared as a predictor and developing the model
    ols_object = smf.ols(formula = 'price~mileage+I(mileage**2)', data = train_updated)
    model = ols_object.fit()
    model.summary()

#Visualizing the regression line with the model consisting of the quadratic transformation of the predictor - mileage
        pred_price = model.predict(train_updated)
        ax = sns.scatterplot(x = 'mileage', y = 'price', data = train_updated, color = 'orange')
        sns.lineplot(x = train_updated.mileage, y = pred_price, color = 'blue')
        plt.xlabel('Mileage')
        plt.ylabel('Predicted car price')
        ax.yaxis.set_major_formatter('${x:,.0f}')
        ax.xaxis.set_major_formatter('{x:,.0f}')

### Cubic Transformation 

#Including mileage squared and mileage cube as predictors and developing the model
    ols_object = smf.ols(formula = 'price~mileage+I(mileage**2)+I(mileage**3)', data = train_updated)
    model = ols_object.fit()
    model.summary()

#Visualizing the model with the cubic transformation of mileage
        pred_price = model.predict(train_updated)
        ax = sns.scatterplot(x = 'mileage', y = 'price', data = train_updated, color = 'orange')
        sns.lineplot(x = train_updated.mileage, y = pred_price, color = 'blue')
        plt.xlabel('Mileage')
        plt.ylabel('Predicted car price')
        ax.yaxis.set_major_formatter('${x:,.0f}')
        ax.xaxis.set_major_formatter('{x:,.0f}')

### Model Assumption

#### MAE (Mean absolute error)

#Computing MAE on test data
pred_price = model.predict(testf)
(np.abs(testp.price - pred_price)).mean()

### Plotting residuals vs fitted values


plt.rcParams["figure.figsize"] = (9,6)
sns.set(font_scale=1.25)
ax = sns.scatterplot(x = model.fittedvalues, y=model.resid,color = 'orange')
sns.lineplot(x = [pred_price.min(),pred_price.max()],y = [0,0],color = 'blue')
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
ax.yaxis.set_major_formatter('${x:,.0f}')
ax.xaxis.set_major_formatter('${x:,.0f}')

### Variable selection
Find the largest subset of predictors in the model developed in the first question *(B.1.1)*, such that their coefficients are zero, i.e., none of the predictors in the subset are statistically significant. 

Does the model $R$-squared change a lot if you remove the set of predictors identifed above from the model in the first question *(B.1.1)*?

**Hint:** You may use the `f_test()` method to test hypotheses.


Predictorsubset = ['lcavol', 'lweight', 'age', 'lbph','svi']

hypothesis = ' = '.join(Predictorsubset) + ' = 0'

f_test = model.f_test(hypothesis)

print(f_test)